# **Código Base de datos por Manzana (Indicadores integrados, VIV + DENUE)**
Esto será el código para crear la base de datos por manzana. <br>
<br>
No toma ninguna otra BD como base, lo crea desde cero, con los datos censales por manzana.
<br>
<!-- Agrupa, además, los resultados por municipio y por zona metropolitana, generando bases de datos adicionales. -->

In [10]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Polygon
from dbfread import DBF
import jenkspy
import json
from jenkspy import jenks_breaks

## Paso dos - Información censal

In [28]:
# #RUTAS ENTRADA - ALEJANDRO (lap)

# #RUTAS CENSOS
# MZNA_2020_salida = r"C:\Users\ADMIN\Desktop\Ejercicio\MZNA_2020_NAC.csv"
# ITER_2020_salida = r"C:\Users\ADMIN\Desktop\Ejercicio\ITER_2020_NAC.csv"
# MZNA_2010_salida = r"C:\Users\ADMIN\Desktop\Ejercicio\MZNA_2010_NAC.csv"
# ITER_2010_salida = r"C:\Users\ADMIN\Desktop\Ejercicio\ITER_2010_NAC.csv"

# # ruta shp base
# SHP_LOC_NAC_ruta = r"C:\Users\Alejandro\Downloads\DATA\SHP\mg_2020_integrado\conjunto_de_datos\00l.shp"
# SHP_MZN_NAC_ruta = r""

# #ruta gpkg fin
# MAPA_TIPO_ruta = r"C:\Users\Alejandro\Downloads\DATA\Tratados\NAC\INT_MZN_NAC\Mzn_Tipologías_NAC.gpkg"


# RUTAS ENTRADA - ALEJANDRO (SSD)

#RUTAS CENSOS
MZNA_2020_ruta = r"Z:\VOCES\DATA\Tratados\NAC\MZNA_2020_NAC.csv"
ITER_2020_ruta = r"Z:\VOCES\DATA\Tratados\NAC\ITER_2020_NAC.csv"
MZNA_2010_ruta = r"Z:\VOCES\DATA\Tratados\NAC\MZNA_2010_NAC.csv"
ITER_2010_ruta = r"Z:\VOCES\DATA\Tratados\NAC\ITER_2010_NAC.csv"

# ruta shp base
SHP_LOC_NAC_ruta = r"Z:\VOCES\DATA\SHP\NAC\00l.shp"
SHP_MZN_NAC_ruta = r"Z:\VOCES\DATA\Tratados\NAC\MZNyLOC_NAC.gpkg"

#ruta gpkg fin
MAPA_TIPO_ruta = r"Z:\VOCES\DATA\Tratados\NAC\INT_MZN_NAC\Mzn_Tipologías_NAC.gpkg"

# Ruta del archivo shp de polígonos de contención urbana
polygons_path = r"Z:\VOCES\DATA\PCU_2018_SHP\PCUS_2018.shp"

#CLAVES METRO
CVE_METRO_ruta = r"Z:\VOCES\DATA\Claves_Metro.csv"

#Claves denue
DENUE_ruta = r"Z:\VOCES\DATA\Tratados\NAC\DENUE.gpkg"

In [ ]:
##########CREAR GPKG BASE

# Leer el archivo de polígonos
polygons_gdf = gpd.read_file(polygons_path)[['geometry', 'CALIF_CLAS']]

# Define el CRS proyectado que deseas utilizar
CRS_PROYECTADO = 'EPSG:6372' 

# Leer shp base
SHP_MZN_gdf= gpd.read_file(SHP_MZN_NAC_ruta, layer='MZN')
SHP_LOC_gdf = gpd.read_file(SHP_LOC_NAC_ruta)

# Proyecta el GeoDataFrame a un CRS proyectado
SHP_MZN_gdf = SHP_MZN_gdf.to_crs(CRS_PROYECTADO)
SHP_LOC_gdf = SHP_LOC_gdf.to_crs(CRS_PROYECTADO)
polygons_gdf = polygons_gdf.to_crs(CRS_PROYECTADO)

# Selección de ITER rural
SHP_LOC_gdf_RURAL = SHP_LOC_gdf[SHP_LOC_gdf['AMBITO'] == 'Rural']

# Filtrar entidades rurales que intersectan con los polígonos
SHP_LOC_gdf_RURAL_INTERSECT = gpd.sjoin(SHP_LOC_gdf_RURAL, polygons_gdf, how='inner', op='intersects')

# Concatenar ITER rural a la capa existente
gdf_concatenated = gpd.GeoDataFrame(pd.concat([SHP_MZN_gdf, SHP_LOC_gdf_RURAL_INTERSECT], ignore_index=True))

# Guardar la capa concatenada en el GeoPackage base
gdf_concatenated.to_file(MAPA_TIPO_ruta, layer='Mapa_Tipologias', driver='GPKG')

# Leer gpkg base
MAPA_TIPO_gdf = gpd.read_file(MAPA_TIPO_ruta, layer='Mapa_Tipologias')

In [15]:
# LEER INFORMACIÓN CENSAL
MZNA_2020_df = pd.read_csv(MZNA_2020_ruta, dtype=str)
ITER_2020_df = pd.read_csv(ITER_2020_ruta, dtype=str)
MZNA_2010_df = pd.read_csv(MZNA_2010_ruta, dtype=str)
ITER_2010_df = pd.read_csv(ITER_2010_ruta, dtype=str)

# Filtrar censos
MZNA_2020_df_F = MZNA_2020_df[['CVEGEO', 'POBTOT', 'VIVTOT', 'TVIVHAB', 'TVIVPAR', 'TVIVPARHAB', 'VIVPAR_HAB', 'VIVPAR_UT', 'OCUPVIVPAR', 'VIVPAR_DES', 'VPH_1CUART', 'VPH_2CUART', 'VPH_3YMASC']]
ITER_2020_df_F = ITER_2020_df[['CVEGEO', 'POBTOT', 'VIVTOT', 'TVIVHAB', 'TVIVPAR', 'TVIVPARHAB', 'VIVPAR_HAB', 'VIVPAR_UT', 'OCUPVIVPAR', 'VIVPAR_DES', 'VPH_1CUART', 'VPH_2CUART', 'VPH_3YMASC']]
MZNA_2010_df_F = MZNA_2010_df[['CVEGEO', 'POBTOT', 'VIVTOT', 'TVIVHAB', 'TVIVPAR', 'TVIVPARHAB', 'VIVPAR_HAB', 'VIVPAR_UT', 'OCUPVIVPAR', 'VIVPAR_DES', 'VPH_1CUART', 'VPH_2CUART', 'VPH_3YMASC']]
ITER_2010_df_F = ITER_2010_df[['CVEGEO', 'POBTOT', 'VIVTOT', 'TVIVHAB', 'TVIVPAR', 'TVIVPARHAB', 'VIVPAR_HAB', 'VIVPAR_UT', 'OCUPVIVPAR', 'VIVPAR_DES', 'VPH_1CUART', 'VPH_2CUART', 'VPH_3YMASC']]

# Renombrar columnas de ITER_2020_df antes del merge para evitar conflictos

MZNA_2020_df_renamed = MZNA_2020_df_F.rename(columns={
    'POBTOT': 'POPTOT_MZNA_2020',
    'VIVTOT': 'VIVTOT_MZNA_2020',
    'TVIVHAB': 'TVIVHAB_MZNA_2020',
    'TVIVPAR': 'TVIVPAR_MZNA_2020',
    'TVIVPARHAB': 'TVIVPARHAB_MZNA_2020',
    'VIVPAR_HAB': 'VIVPAR_HAB_MZNA_2020',
    'VIVPAR_UT': 'VIVPAR_UT_MZNA_2020',
    'OCUPVIVPAR': 'OCUPVIVPAR_MZNA_2020',
    'VIVPAR_DES': 'VIVPAR_DES_MZNA_2020',
    'VPH_1CUART': 'VIV1C_MZNA_2020',
    'VPH_2CUART': 'VIV2C_MZNA_2020',
    'VPH_3YMASC': 'VIV3C_MZNA_2020'
})

ITER_2020_df_renamed = ITER_2020_df_F.rename(columns={
    'POBTOT': 'POPTOT_ITER_2020',
    'VIVTOT': 'VIVTOT_ITER_2020',
    'TVIVHAB': 'TVIVHAB_ITER_2020',
    'TVIVPAR': 'TVIVPAR_ITER_2020',
    'TVIVPARHAB': 'TVIVPARHAB_ITER_2020',
    'VIVPAR_HAB': 'VIVPAR_HAB_ITER_2020',
    'VIVPAR_UT': 'VIVPAR_UT_ITER_2020',
    'OCUPVIVPAR': 'OCUPVIVPAR_ITER_2020',
    'VIVPAR_DES': 'VIVPAR_DES_ITER_2020',
    'VPH_1CUART': 'VIV1C_ITER_2020',
    'VPH_2CUART': 'VIV2C_ITER_2020',
    'VPH_3YMASC': 'VIV3C_ITER_2020'
})

MZNA_2010_df_renamed = MZNA_2010_df_F.rename(columns={
    'POBTOT': 'POPTOT_MZNA_2010',
    'VIVTOT': 'VIVTOT_MZNA_2010',
    'TVIVHAB': 'TVIVHAB_MZNA_2010',
    'TVIVPAR': 'TVIVPAR_MZNA_2010',
    'TVIVPARHAB': 'TVIVPARHAB_MZNA_2010',
    'VIVPAR_HAB': 'VIVPAR_HAB_MZNA_2010',
    'VIVPAR_UT': 'VIVPAR_UT_MZNA_2010',
    'OCUPVIVPAR': 'OCUPVIVPAR_MZNA_2010',
    'VIVPAR_DES': 'VIVPAR_DES_MZNA_2010',
    'VPH_1CUART': 'VIV1C_MZNA_2010',
    'VPH_2CUART': 'VIV2C_MZNA_2010',
    'VPH_3YMASC': 'VIV3C_MZNA_2010'
})

ITER_2010_df_renamed = ITER_2010_df_F.rename(columns={
    'POBTOT': 'POPTOT_ITER_2010',
    'VIVTOT': 'VIVTOT_ITER_2010',
    'TVIVHAB': 'TVIVHAB_ITER_2010',
    'TVIVPAR': 'TVIVPAR_ITER_2010',
    'TVIVPARHAB': 'TVIVPARHAB_ITER_2010',
    'VIVPAR_HAB': 'VIVPAR_HAB_ITER_2010',
    'VIVPAR_UT': 'VIVPAR_UT_ITER_2010',
    'OCUPVIVPAR': 'OCUPVIVPAR_ITER_2010',
    'VIVPAR_DES': 'VIVPAR_DES_ITER_2010',
    'VPH_1CUART': 'VIV1C_ITER_2010',
    'VPH_2CUART': 'VIV2C_ITER_2010',
    'VPH_3YMASC': 'VIV3C_ITER_2010'
})

MAPA_TIPO_gdf['CVEGEO'] = MAPA_TIPO_gdf['CVEGEO'].astype(str)
MZNA_2020_df_renamed['CVEGEO'] = MZNA_2020_df_renamed['CVEGEO'].astype(str)
ITER_2020_df_renamed['CVEGEO'] = ITER_2020_df_renamed['CVEGEO'].astype(str)
MZNA_2010_df_renamed['CVEGEO'] = MZNA_2010_df_renamed['CVEGEO'].astype(str)
ITER_2010_df_renamed['CVEGEO'] = ITER_2010_df_renamed['CVEGEO'].astype(str)

In [18]:
# Unir campos de los DataFrames al GeoPackage base por CVEGEO
MAPA_TIPO_gdf1 = MAPA_TIPO_gdf.merge(
    MZNA_2020_df_renamed[[
        'CVEGEO', 'POPTOT_MZNA_2020', 'VIVTOT_MZNA_2020', 'TVIVHAB_MZNA_2020', 'TVIVPAR_MZNA_2020', 'TVIVPARHAB_MZNA_2020', 'VIVPAR_HAB_MZNA_2020', 'VIVPAR_UT_MZNA_2020', 'OCUPVIVPAR_MZNA_2020', 'VIVPAR_DES_MZNA_2020', 'VIV1C_MZNA_2020', 'VIV2C_MZNA_2020', 'VIV3C_MZNA_2020']],
    on='CVEGEO', how='left')

MAPA_TIPO_gdf2 = MAPA_TIPO_gdf1.merge(
    ITER_2020_df_renamed[[
        'CVEGEO', 'POPTOT_ITER_2020', 'VIVTOT_ITER_2020', 'TVIVHAB_ITER_2020', 'TVIVPAR_ITER_2020', 'TVIVPARHAB_ITER_2020', 'VIVPAR_HAB_ITER_2020', 'VIVPAR_UT_ITER_2020', 'OCUPVIVPAR_ITER_2020', 'VIVPAR_DES_ITER_2020', 'VIV1C_ITER_2020', 'VIV2C_ITER_2020', 'VIV3C_ITER_2020']],
    on='CVEGEO', how='left')


MAPA_TIPO_gdf3 = MAPA_TIPO_gdf2.merge(
    MZNA_2010_df_renamed[[
        'CVEGEO', 'POPTOT_MZNA_2010', 'VIVTOT_MZNA_2010', 'TVIVHAB_MZNA_2010', 'TVIVPAR_MZNA_2010', 'TVIVPARHAB_MZNA_2010', 'VIVPAR_HAB_MZNA_2010', 'VIVPAR_UT_MZNA_2010', 'OCUPVIVPAR_MZNA_2010', 'VIVPAR_DES_MZNA_2010', 'VIV1C_MZNA_2010', 'VIV2C_MZNA_2010', 'VIV3C_MZNA_2010']],
    on='CVEGEO', how='left')

MAPA_TIPO_gdf4 = MAPA_TIPO_gdf3.merge(
    ITER_2010_df_renamed[[
        'CVEGEO', 'POPTOT_ITER_2010', 'VIVTOT_ITER_2010', 'TVIVHAB_ITER_2010', 'TVIVPAR_ITER_2010', 'TVIVPARHAB_ITER_2010', 'VIVPAR_HAB_ITER_2010', 'VIVPAR_UT_ITER_2010', 'OCUPVIVPAR_ITER_2010', 'VIVPAR_DES_ITER_2010', 'VIV1C_ITER_2010', 'VIV2C_ITER_2010', 'VIV3C_ITER_2010']],
    on='CVEGEO', how='left')

# Reemplazar * con 1
def replace(dataframe):
    return dataframe.replace('*', '1')

# Limpiar *
MAPA_TIPO_gdf4_A = replace(MAPA_TIPO_gdf4)

# Reemplazar N/D
MAPA_TIPO_gdf4_B = MAPA_TIPO_gdf4_A.replace('N/D', np.nan)

# Actualizar valores sin información de "null" a 0
MAPA_TIPO_gdf4_C = MAPA_TIPO_gdf4_B.fillna(0)

In [26]:
# Crear campos POPTOT2010, POPTOT2020, VIVPAR2010 y VIVPAR2020 y llenado respectivamente
MAPA_TIPO_gdf4_C['POPTOT2010'] = MAPA_TIPO_gdf4_C['POPTOT_MZNA_2010'].astype(int) + MAPA_TIPO_gdf4_C['POPTOT_ITER_2010'].astype(int)
MAPA_TIPO_gdf4_C['POPTOT2020'] = MAPA_TIPO_gdf4_C['POPTOT_MZNA_2020'].astype(int) + MAPA_TIPO_gdf4_C['POPTOT_ITER_2020'].astype(int)
MAPA_TIPO_gdf4_C['VIVTOT2010'] = MAPA_TIPO_gdf4_C['VIVTOT_MZNA_2010'].astype(int) +  MAPA_TIPO_gdf4_C['VIVTOT_ITER_2010'].astype(int)
MAPA_TIPO_gdf4_C['VIVTOT2020'] = MAPA_TIPO_gdf4_C['VIVTOT_MZNA_2020'].astype(int) +  MAPA_TIPO_gdf4_C['VIVTOT_ITER_2020'].astype(int)
MAPA_TIPO_gdf4_C['TVIVHAB2010'] = MAPA_TIPO_gdf4_C['TVIVHAB_MZNA_2010'].astype(int) +  MAPA_TIPO_gdf4_C['TVIVHAB_ITER_2010'].astype(int)
MAPA_TIPO_gdf4_C['TVIVHAB2020'] = MAPA_TIPO_gdf4_C['TVIVHAB_MZNA_2020'].astype(int) +  MAPA_TIPO_gdf4_C['TVIVHAB_ITER_2020'].astype(int)
MAPA_TIPO_gdf4_C['TVIVPAR2010'] = MAPA_TIPO_gdf4_C['TVIVPAR_MZNA_2010'].astype(int) +  MAPA_TIPO_gdf4_C['TVIVPAR_ITER_2010'].astype(int)
MAPA_TIPO_gdf4_C['TVIVPAR2020'] = MAPA_TIPO_gdf4_C['TVIVPAR_MZNA_2020'].astype(int) +  MAPA_TIPO_gdf4_C['TVIVPAR_ITER_2020'].astype(int)
MAPA_TIPO_gdf4_C['TVIVPARHAB2010'] = MAPA_TIPO_gdf4_C['TVIVPARHAB_MZNA_2010'].astype(int) +  MAPA_TIPO_gdf4_C['TVIVPARHAB_ITER_2010'].astype(int)
MAPA_TIPO_gdf4_C['TVIVPARHAB2020'] = MAPA_TIPO_gdf4_C['TVIVPARHAB_MZNA_2020'].astype(int) +  MAPA_TIPO_gdf4_C['TVIVPARHAB_ITER_2020'].astype(int)
MAPA_TIPO_gdf4_C['VIVPAR_HAB2010'] = MAPA_TIPO_gdf4_C['VIVPAR_HAB_MZNA_2010'].astype(int) + MAPA_TIPO_gdf4_C['VIVPAR_HAB_ITER_2010'].astype(int)
MAPA_TIPO_gdf4_C['VIVPAR_HAB2020'] = MAPA_TIPO_gdf4_C['VIVPAR_HAB_MZNA_2020'].astype(int) + MAPA_TIPO_gdf4_C['VIVPAR_HAB_ITER_2020'].astype(int)
MAPA_TIPO_gdf4_C['VIVPAR_UT2010'] = MAPA_TIPO_gdf4_C['VIVPAR_UT_MZNA_2010'].astype(int) + MAPA_TIPO_gdf4_C['VIVPAR_UT_ITER_2010'].astype(int)
MAPA_TIPO_gdf4_C['VIVPAR_UT2020'] = MAPA_TIPO_gdf4_C['VIVPAR_UT_MZNA_2020'].astype(int) + MAPA_TIPO_gdf4_C['VIVPAR_UT_ITER_2020'].astype(int)
MAPA_TIPO_gdf4_C['OCUPVIVPAR2010'] = MAPA_TIPO_gdf4_C['OCUPVIVPAR_MZNA_2010'].astype(int) +  MAPA_TIPO_gdf4_C['OCUPVIVPAR_ITER_2010'].astype(int)
MAPA_TIPO_gdf4_C['OCUPVIVPAR2020'] = MAPA_TIPO_gdf4_C['OCUPVIVPAR_MZNA_2020'].astype(int) +  MAPA_TIPO_gdf4_C['OCUPVIVPAR_ITER_2020'].astype(int)
MAPA_TIPO_gdf4_C['VIVPAR_DES2010'] = MAPA_TIPO_gdf4_C['VIVPAR_DES_MZNA_2010'].astype(int) + MAPA_TIPO_gdf4_C['VIVPAR_DES_ITER_2010'].astype(int)
MAPA_TIPO_gdf4_C['VIVPAR_DES2020'] = MAPA_TIPO_gdf4_C['VIVPAR_DES_MZNA_2020'].astype(int) + MAPA_TIPO_gdf4_C['VIVPAR_DES_ITER_2020'].astype(int)
MAPA_TIPO_gdf4_C['VIV1C2010'] = MAPA_TIPO_gdf4_C['VIV1C_MZNA_2010'].astype(int) + MAPA_TIPO_gdf4_C['VIV1C_ITER_2010'].astype(int)
MAPA_TIPO_gdf4_C['VIV1C2020'] = MAPA_TIPO_gdf4_C['VIV1C_MZNA_2020'].astype(int) + MAPA_TIPO_gdf4_C['VIV1C_ITER_2020'].astype(int)
MAPA_TIPO_gdf4_C['VIV2C2010'] = MAPA_TIPO_gdf4_C['VIV2C_MZNA_2010'].astype(int) + MAPA_TIPO_gdf4_C['VIV2C_ITER_2010'].astype(int)
MAPA_TIPO_gdf4_C['VIV2C2020'] = MAPA_TIPO_gdf4_C['VIV2C_MZNA_2020'].astype(int) + MAPA_TIPO_gdf4_C['VIV2C_ITER_2020'].astype(int)
MAPA_TIPO_gdf4_C['VIV3C2010'] = MAPA_TIPO_gdf4_C['VIV3C_MZNA_2010'].astype(int) + MAPA_TIPO_gdf4_C['VIV3C_ITER_2010'].astype(int)
MAPA_TIPO_gdf4_C['VIV3C2020'] = MAPA_TIPO_gdf4_C['VIV3C_MZNA_2020'].astype(int) + MAPA_TIPO_gdf4_C['VIV3C_ITER_2020'].astype(int)

# Convertir campos a enteros
fields_to_convert = ['POPTOT2010', 'POPTOT2020', 'VIVTOT2010', 'VIVTOT2020', 
    'TVIVHAB2010', 'TVIVHAB2020', 'TVIVPAR2010', 'TVIVPAR2020',
    'TVIVPARHAB2010', 'TVIVPARHAB2020', 'VIVPAR_HAB2010', 'VIVPAR_HAB2020',
    'VIVPAR_UT2010', 'VIVPAR_UT2020', 'OCUPVIVPAR2010', 'OCUPVIVPAR2020',
    'VIVPAR_DES2010', 'VIVPAR_DES2020', 'VIV1C2010', 'VIV1C2020', 
    'VIV2C2010', 'VIV2C2020', 'VIV3C2010', 'VIV3C2020']
for field in fields_to_convert:
    MAPA_TIPO_gdf4_C[field] = MAPA_TIPO_gdf4_C[field].astype(int)

# Reordenar columnas
ordered_columns = [
    'CVEGEO', 'POPTOT2010', 'POPTOT2020', 'VIVTOT2010', 'VIVTOT2020', 
    'TVIVHAB2010', 'TVIVHAB2020', 'TVIVPAR2010', 'TVIVPAR2020',
    'TVIVPARHAB2010', 'TVIVPARHAB2020', 'VIVPAR_HAB2010', 'VIVPAR_HAB2020',
    'VIVPAR_UT2010', 'VIVPAR_UT2020', 'OCUPVIVPAR2010', 'OCUPVIVPAR2020',
    'VIVPAR_DES2010', 'VIVPAR_DES2020', 'VIV1C2010', 'VIV1C2020', 
    'VIV2C2010', 'VIV2C2020', 'VIV3C2010', 'VIV3C2020'
] + [col for col in MAPA_TIPO_gdf4.columns if col not in [
    'CVEGEO', 'POPTOT2010', 'POPTOT2020', 'VIVTOT2010', 'VIVTOT2020', 
    'TVIVHAB2010', 'TVIVHAB2020', 'TVIVPAR2010', 'TVIVPAR2020',
    'TVIVPARHAB2010', 'TVIVPARHAB2020', 'VIVPAR_HAB2010', 'VIVPAR_HAB2020',
    'VIVPAR_UT2010', 'VIVPAR_UT2020', 'OCUPVIVPAR2010', 'OCUPVIVPAR2020',
    'VIVPAR_DES2010', 'VIVPAR_DES2020', 'VIV1C2010', 'VIV1C2020', 
    'VIV2C2010', 'VIV2C2020', 'VIV3C2010', 'VIV3C2020'
]]


MAPA_TIPO_gdf4_C = MAPA_TIPO_gdf4_C[ordered_columns]

# Guardar el GeoPackage base actualizado
MAPA_TIPO_gdf4_C.to_file(MAPA_TIPO_ruta, layer='BASE_DATOS_NAC_MZN', driver="GPKG")

In [29]:
#AGRUPAR Y ASIGNAR FILTRO Y PROM POR ZM

#Subir archivo con CVE_METRO
CVE_METRO_df = pd.read_csv(CVE_METRO_ruta, dtype={'CVE_MET': str, 'CVEMUN':str})

# # Leer la capa existente del GeoPackage base
# MAPA_TIPO_gdf = gpd.read_file(MAPA_TIPO_ruta, layer='BASE_DATOS_NAC_MZN')

#Crear campo CVEMUN
def homologate_municipio_id(row):
    return f"{row['CVE_ENT']}{row['CVE_MUN']}"

# MAPA_TIPO_gdf['CVEMUN'] = MAPA_TIPO_gdf.apply(homologate_municipio_id, axis=1)
MAPA_TIPO_gdf4_C['CVEMUN'] = MAPA_TIPO_gdf4_C.apply(homologate_municipio_id, axis=1)

C:\Users\UTIC\anaconda3\Lib\site-packages\geopandas\geodataframe.py:1525: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [30]:
#Añadir claves a GPKG
# MAPA_TIPO_gdf = MAPA_TIPO_gdf.merge(CVE_METRO_df[['CVEMUN', 'CVE_MET', 'NOM_MET', 'TIPO_MET','TIPO_MUN']], on='CVEMUN', how='left')
MAPA_TIPO_gdf4_C = MAPA_TIPO_gdf4_C.merge(CVE_METRO_df[['CVEMUN', 'CVE_MET', 'NOM_MET', 'TIPO_MET','TIPO_MUN']], on='CVEMUN', how='left')

#FILTRAR MUNICIPIOS ZONAS METROPOLITANAS
valores_filtro = CVE_METRO_df['CVEMUN'].unique()

# MAPA_TIPO_gdf = MAPA_TIPO_gdf[(MAPA_TIPO_gdf['CVEMUN'].isin(valores_filtro))]
MAPA_TIPO_gdf4_C = MAPA_TIPO_gdf4_C[(MAPA_TIPO_gdf4_C['CVEMUN'].isin(valores_filtro))]

# Guardar el GeoPackage base actualizado
# MAPA_TIPO_gdf.to_file(MAPA_TIPO_ruta, layer= 'BASE_DATOS_METRO_MZN', driver="GPKG")
MAPA_TIPO_gdf4_C.to_file(MAPA_TIPO_ruta, layer= 'BASE_DATOS_METRO_MZN', driver="GPKG")

In [8]:
# Leer la capa existente del GeoPackage base
MAPA_TIPO_gdf = gpd.read_file(MAPA_TIPO_ruta, layer='BASE_DATOS_METRO_MZN')

In [32]:
# Creación de campos y llenado de información

MAPA_TIPO_gdf4_C['area_ha'] = MAPA_TIPO_gdf4_C.area / 10000
MAPA_TIPO_gdf4_C['DESHABITACION'] = MAPA_TIPO_gdf4_C['VIVPAR_DES2020'] / MAPA_TIPO_gdf4_C['VIVTOT2020']
MAPA_TIPO_gdf4_C['DIFPOP'] = MAPA_TIPO_gdf4_C['POPTOT2020'] - MAPA_TIPO_gdf4_C['POPTOT2010']
MAPA_TIPO_gdf4_C['DIFVIV'] = MAPA_TIPO_gdf4_C['VIVTOT2020'] - MAPA_TIPO_gdf4_C['VIVTOT2010']
MAPA_TIPO_gdf4_C['RELPOP'] = MAPA_TIPO_gdf4_C['DIFPOP'] / MAPA_TIPO_gdf4_C['POPTOT2010']
MAPA_TIPO_gdf4_C['RELVIV'] = MAPA_TIPO_gdf4_C['DIFVIV'] / MAPA_TIPO_gdf4_C['VIVTOT2010']
MAPA_TIPO_gdf4_C['CVEPOP'] = np.where(MAPA_TIPO_gdf4_C['RELPOP'] < -0.000000001, 'A', 'B')
MAPA_TIPO_gdf4_C['CVEVIV'] = np.where(MAPA_TIPO_gdf4_C['RELVIV'] < -.0703, '1',
                                   np.where((MAPA_TIPO_gdf4_C['RELVIV'] >= -.0703) & (MAPA_TIPO_gdf4_C['RELVIV'] <= .0703), 
                                            '2', '3'))
MAPA_TIPO_gdf4_C['CVE'] = MAPA_TIPO_gdf4_C['CVEPOP'] + MAPA_TIPO_gdf4_C['CVEVIV'].astype(str)
conditions = [
    (MAPA_TIPO_gdf4_C['POPTOT2020'] == 0),
    (MAPA_TIPO_gdf4_C['POPTOT2010'] == 0) & (MAPA_TIPO_gdf4_C['VIVTOT2010'] == 0) & (MAPA_TIPO_gdf4_C['CVE'] == 'B3')
]
choices = ['DH', 'B3\'']
MAPA_TIPO_gdf4_C['TIPOLOGIA'] = np.select(conditions, choices, default=MAPA_TIPO_gdf4_C['CVE'])

MAPA_TIPO_gdf4_C['DENS_POP10'] = MAPA_TIPO_gdf4_C['POPTOT2010'] / MAPA_TIPO_gdf4_C['area_ha']
MAPA_TIPO_gdf4_C['DENS_POP20'] = MAPA_TIPO_gdf4_C['POPTOT2020'] / MAPA_TIPO_gdf4_C['area_ha']

MAPA_TIPO_gdf4_C['DENS_VIV10'] = MAPA_TIPO_gdf4_C['VIVTOT2010'] / MAPA_TIPO_gdf4_C['area_ha']
MAPA_TIPO_gdf4_C['DENS_VIV20'] = MAPA_TIPO_gdf4_C['VIVTOT2020'] / MAPA_TIPO_gdf4_C['area_ha']

MAPA_TIPO_gdf4_C['CUARTOS2010'] = ((MAPA_TIPO_gdf4_C['VIV1C2010']*1) + (MAPA_TIPO_gdf4_C['VIV2C2010']*2) + (MAPA_TIPO_gdf4_C['VIV3C2010']*4))
MAPA_TIPO_gdf4_C['CUARTOS2020'] = ((MAPA_TIPO_gdf4_C['VIV1C2020']*1) + (MAPA_TIPO_gdf4_C['VIV2C2020']*2) + (MAPA_TIPO_gdf4_C['VIV3C2020']*4))

MAPA_TIPO_gdf4_C['CPERC2010'] = MAPA_TIPO_gdf4_C['CUARTOS2010'] / MAPA_TIPO_gdf4_C['POPTOT2010']
MAPA_TIPO_gdf4_C['CPERC2020'] = MAPA_TIPO_gdf4_C['CUARTOS2020'] / MAPA_TIPO_gdf4_C['POPTOT2020']

MAPA_TIPO_gdf4_C['HABXVIV2010'] = MAPA_TIPO_gdf4_C['POPTOT2010'] / MAPA_TIPO_gdf4_C['VIVTOT2010']
MAPA_TIPO_gdf4_C['HABXVIV2020'] = MAPA_TIPO_gdf4_C['POPTOT2020'] / MAPA_TIPO_gdf4_C['VIVTOT2020']

def asignar_subtipologia(row):
    # Valores específicos para deshabitación
    rangos_deshabitacion_grupo = [0, .0703, .1416, .2812]  # Modifica estos valores según tus necesidades
    
# Asignar subtipología según los rangos definidos
    if rangos_deshabitacion_grupo[0] <= row['DESHABITACION'] < rangos_deshabitacion_grupo[1]:
        return 'a'
    elif rangos_deshabitacion_grupo[1] <= row['DESHABITACION'] < rangos_deshabitacion_grupo[2]:
        return 'b'
    elif rangos_deshabitacion_grupo[2] <= row['DESHABITACION'] < rangos_deshabitacion_grupo[3]:
        return 'c'
    elif row['DESHABITACION'] >= rangos_deshabitacion_grupo[3]:
        return 'd'
    else:
        return '0'  # Valor predeterminado

# Crear un nuevo campo 'R_DESHAB' y llenarlo con las subtipologías asignadas
MAPA_TIPO_gdf4_C['R_DESHAB'] = MAPA_TIPO_gdf4_C.apply(asignar_subtipologia, axis=1)

# Crear el campo SUBTIPOLOGÍA
MAPA_TIPO_gdf4_C['SUBTIPOLOGÍA'] = MAPA_TIPO_gdf4_C['TIPOLOGIA'].astype(str) + '-' + MAPA_TIPO_gdf4_C['R_DESHAB'].astype(str)

# Guardar el GeoPackage con los nuevos campos
MAPA_TIPO_gdf4_C.to_file(MAPA_TIPO_ruta, layer='Mzn_Subtipologias', driver='GPKG')

C:\Users\UTIC\anaconda3\Lib\site-packages\geopandas\geodataframe.py:1525: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)
C:\Users\UTIC\anaconda3\Lib\site-packages\geopandas\geodataframe.py:1525: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)
C:\Users\UTIC\anaconda3\Lib\site-packages\geopandas\geodataframe.py:1525: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider 

In [ ]:
# # Creación de campos y llenado de información

# MAPA_TIPO_gdf['area_ha'] = MAPA_TIPO_gdf.area / 10000
# MAPA_TIPO_gdf['DESHABITACION'] = MAPA_TIPO_gdf['VIVPAR_DES2020'] / MAPA_TIPO_gdf['VIVTOT2020']
# MAPA_TIPO_gdf['DIFPOP'] = MAPA_TIPO_gdf['POPTOT2020'] - MAPA_TIPO_gdf['POPTOT2010']
# MAPA_TIPO_gdf['DIFVIV'] = MAPA_TIPO_gdf['VIVTOT2020'] - MAPA_TIPO_gdf['VIVTOT2010']
# MAPA_TIPO_gdf['RELPOP'] = MAPA_TIPO_gdf['DIFPOP'] / MAPA_TIPO_gdf['POPTOT2010']
# MAPA_TIPO_gdf['RELVIV'] = MAPA_TIPO_gdf['DIFVIV'] / MAPA_TIPO_gdf['VIVTOT2010']
# MAPA_TIPO_gdf['CVEPOP'] = np.where(MAPA_TIPO_gdf['RELPOP'] < -0.000000001, 'A', 'B')
# MAPA_TIPO_gdf['CVEVIV'] = np.where(MAPA_TIPO_gdf['RELVIV'] < -.0703, '1',
#                                    np.where((MAPA_TIPO_gdf['RELVIV'] >= -.0703) & (MAPA_TIPO_gdf['RELVIV'] <= .0703), 
#                                             '2', '3'))
# MAPA_TIPO_gdf['CVE'] = MAPA_TIPO_gdf['CVEPOP'] + MAPA_TIPO_gdf['CVEVIV'].astype(str)
# conditions = [
#     (MAPA_TIPO_gdf['POPTOT2020'] == 0),
#     (MAPA_TIPO_gdf['POPTOT2010'] == 0) & (MAPA_TIPO_gdf['VIVTOT2010'] == 0) & (MAPA_TIPO_gdf['CVE'] == 'B3')
# ]
# choices = ['DH', 'B3\'']
# MAPA_TIPO_gdf['TIPOLOGIA'] = np.select(conditions, choices, default=MAPA_TIPO_gdf['CVE'])

# MAPA_TIPO_gdf['DENS_POP10'] = MAPA_TIPO_gdf['POPTOT2010'] / MAPA_TIPO_gdf['area_ha']
# MAPA_TIPO_gdf['DENS_POP20'] = MAPA_TIPO_gdf['POPTOT2020'] / MAPA_TIPO_gdf['area_ha']

# MAPA_TIPO_gdf['DENS_VIV10'] = MAPA_TIPO_gdf['VIVTOT2010'] / MAPA_TIPO_gdf['area_ha']
# MAPA_TIPO_gdf['DENS_VIV20'] = MAPA_TIPO_gdf['VIVTOT2020'] / MAPA_TIPO_gdf['area_ha']

# MAPA_TIPO_gdf['CUARTOS2010'] = ((MAPA_TIPO_gdf['VIV1C2010']*1) + (MAPA_TIPO_gdf['VIV2C2010']*2) + (MAPA_TIPO_gdf['VIV3C2010']*4))
# MAPA_TIPO_gdf['CUARTOS2020'] = ((MAPA_TIPO_gdf['VIV1C2020']*1) + (MAPA_TIPO_gdf['VIV2C2020']*2) + (MAPA_TIPO_gdf['VIV3C2020']*4))

# MAPA_TIPO_gdf['CPERC2010'] = MAPA_TIPO_gdf['CUARTOS2010'] / MAPA_TIPO_gdf['POPTOT2010']
# MAPA_TIPO_gdf['CPERC2020'] = MAPA_TIPO_gdf['CUARTOS2020'] / MAPA_TIPO_gdf['POPTOT2020']

# MAPA_TIPO_gdf['HABXVIV2010'] = MAPA_TIPO_gdf['POPTOT2010'] / MAPA_TIPO_gdf['VIVTOT2010']
# MAPA_TIPO_gdf['HABXVIV2020'] = MAPA_TIPO_gdf['POPTOT2020'] / MAPA_TIPO_gdf['VIVTOT2020']

# def asignar_subtipologia(row):
#     # Valores específicos para deshabitación
#     rangos_deshabitacion_grupo = [0, .0703, .1416, .2812]  # Modifica estos valores según tus necesidades
    
# # Asignar subtipología según los rangos definidos
#     if rangos_deshabitacion_grupo[0] <= row['DESHABITACION'] < rangos_deshabitacion_grupo[1]:
#         return 'a'
#     elif rangos_deshabitacion_grupo[1] <= row['DESHABITACION'] < rangos_deshabitacion_grupo[2]:
#         return 'b'
#     elif rangos_deshabitacion_grupo[2] <= row['DESHABITACION'] < rangos_deshabitacion_grupo[3]:
#         return 'c'
#     elif row['DESHABITACION'] >= rangos_deshabitacion_grupo[3]:
#         return 'd'
#     else:
#         return '0'  # Valor predeterminado

# # Crear un nuevo campo 'R_DESHAB' y llenarlo con las subtipologías asignadas
# MAPA_TIPO_gdf['R_DESHAB'] = MAPA_TIPO_gdf.apply(asignar_subtipologia, axis=1)

# # Crear el campo SUBTIPOLOGÍA
# MAPA_TIPO_gdf['SUBTIPOLOGÍA'] = MAPA_TIPO_gdf['TIPOLOGIA'].astype(str) + '-' + MAPA_TIPO_gdf['R_DESHAB'].astype(str)

# # Guardar el GeoPackage con los nuevos campos
# MAPA_TIPO_gdf.to_file(MAPA_TIPO_ruta, layer='Mzn_Subtipologias', driver='GPKG')


In [ ]:
## datos DENUE

In [ ]:
# Cargar los GeoDataFrames de las manzanas y las unidades económicas
MAPA_TIPO_gdf = gpd.read_file(MAPA_TIPO_ruta, layer='Mzn_Subtipologias')

SHP_DENUE_2020_gdf = gpd.read_file(DENUE_ruta, layer='2020')
SHP_DENUE_2010_gdf = gpd.read_file(DENUE_ruta, layer='2010')

In [ ]:
# Verificar que ambos datasets tienen el mismo sistema de coordenadas
if MAPA_TIPO_gdf.crs != SHP_DENUE_2020_gdf.crs:
    SHP_DENUE_2020_gdf = SHP_DENUE_2020_gdf.to_crs(MAPA_TIPO_gdf.crs)
if MAPA_TIPO_gdf.crs != SHP_DENUE_2010_gdf.crs:
    SHP_DENUE_2010_gdf = SHP_DENUE_2010_gdf.to_crs(MAPA_TIPO_gdf.crs)

# Función para aplicar buffer, contar unidades económicas y sumar empleados dentro de los polígonos
def calcular_datos_con_buffer(manzanas, ue_data, col_ue, col_emp, buffer_dist=2):
    # Crear un buffer de 2 metros alrededor de cada polígono de las localidades
    manzanas_buffered = manzanas.copy()
    manzanas_buffered['geometry'] = manzanas_buffered.geometry.buffer(buffer_dist)

    # Realizar un 'spatial join' para unir los puntos con los polígonos en los que caen
    join = gpd.sjoin(ue_data, manzanas_buffered, how="inner", predicate='within', rsuffix='_manzana')

    # Calcular el conteo de unidades económicas y la suma de empleados por polígono
    conteo_ue = join.groupby('CVEGEO').size()  # Cuenta las unidades económicas por CVEGEO
    suma_emp = join.groupby('CVEGEO')['num_emp'].sum()  # Suma los empleados por CVEGEO

    # Asignar los resultados al geodataframe de localidades usando CVEGEO como clave única
    manzanas[col_ue] = manzanas['CVEGEO'].map(conteo_ue).fillna(0).astype(int)
    manzanas[col_emp] = manzanas['CVEGEO'].map(suma_emp).fillna(0).astype(int)

# Calcular datos aplicando buffer de 2 metros
calcular_datos_con_buffer(MAPA_TIPO_gdf, SHP_DENUE_2020_gdf, 'UE_20', 'Emp_20', buffer_dist=2)
calcular_datos_con_buffer(MAPA_TIPO_gdf, SHP_DENUE_2010_gdf, 'UE_10', 'Emp_10', buffer_dist=2)


In [ ]:
#Tamaño de unidades promedio: Empleados entre UE por manzana
MAPA_TIPO_gdf['TAMAÑO'] = MAPA_TIPO_gdf['Emp_20'] / MAPA_TIPO_gdf['UE_20']

# Calcular las sumas de POPTOT2020 y POPTOT2010 por CVE_MET
sum_emptot2020 = MAPA_TIPO_gdf.groupby('CVEMUN')['Emp_20'].transform('sum')
sum_emptot2010 = MAPA_TIPO_gdf.groupby('CVEMUN')['Emp_10'].transform('sum')

# Calcular TASA_POP_MUN
MAPA_TIPO_gdf['TASA_EMP_MUN'] = (((sum_emptot2020 / sum_emptot2010) ** (1/10)) - 1) * 100
print(8)
#resto de campos
MAPA_TIPO_gdf['DIF_EMP'] = MAPA_TIPO_gdf['Emp_20'] - MAPA_TIPO_gdf['Emp_10']
MAPA_TIPO_gdf['DIF_UEC'] = MAPA_TIPO_gdf['UE_20'] - MAPA_TIPO_gdf['UE_10']
MAPA_TIPO_gdf['REL_EMP'] = MAPA_TIPO_gdf['DIF_EMP'] / MAPA_TIPO_gdf['Emp_10']
MAPA_TIPO_gdf['REL_UEC'] = MAPA_TIPO_gdf['DIF_UEC'] / MAPA_TIPO_gdf['UE_10']
MAPA_TIPO_gdf['CVE_EMP'] = np.where(MAPA_TIPO_gdf['REL_EMP'] < -0.000000001, '1', '2')
MAPA_TIPO_gdf['CVE_UEC'] = np.where(MAPA_TIPO_gdf['REL_UEC'] < -.1, 'A',
                                   np.where((MAPA_TIPO_gdf['REL_UEC'] >= -.1) & (MAPA_TIPO_gdf['REL_UEC'] <= .1), 
                                            'B', 'C'))
MAPA_TIPO_gdf['CVE_ue'] = MAPA_TIPO_gdf['CVE_UEC'].astype(str) + MAPA_TIPO_gdf['CVE_EMP']
conditions = [
    (MAPA_TIPO_gdf['Emp_20'] == 0),
    (MAPA_TIPO_gdf['Emp_10'] == 0) & (MAPA_TIPO_gdf['UE_10'] == 0) & (MAPA_TIPO_gdf['CVE_ue'] == 'C2')
]
choices = ['00', 'C2+']
MAPA_TIPO_gdf['TIPOLOGIA_ue'] = np.select(conditions, choices, default=MAPA_TIPO_gdf['CVE_ue'])

MAPA_TIPO_gdf['DENS_EMP20'] = MAPA_TIPO_gdf['Emp_20'] / MAPA_TIPO_gdf['area_ha']
MAPA_TIPO_gdf['DENS_EMP10'] = MAPA_TIPO_gdf['Emp_10'] / MAPA_TIPO_gdf['area_ha']

MAPA_TIPO_gdf['DENS_UEC20'] = MAPA_TIPO_gdf['UE_20'] / MAPA_TIPO_gdf['area_ha']
MAPA_TIPO_gdf['DENS_UEC10'] = MAPA_TIPO_gdf['UE_10'] / MAPA_TIPO_gdf['area_ha']


def asignar_subtipologia(row):
    # Valores específicos para deshabitación
    rangos_deshabitacion_grupo = [0, 1, 30, 100]  # Modifica estos valores según tus necesidades
    
# Asignar subtipología según los rangos definidos
    if rangos_deshabitacion_grupo[0] <= row['TAMAÑO'] < rangos_deshabitacion_grupo[1]:
        return 'SUE'
    elif rangos_deshabitacion_grupo[1] <= row['TAMAÑO'] < rangos_deshabitacion_grupo[2]:
        return 'MICRO'
    elif rangos_deshabitacion_grupo[2] <= row['TAMAÑO'] < rangos_deshabitacion_grupo[3]:
        return 'MEDIANA'
    elif row['TAMAÑO'] >= rangos_deshabitacion_grupo[3]:
        return 'GRANDE'
    else:
        return '0'  # Valor predeterminado
print(11)

# Crear un nuevo campo 'R_DESHAB' y llenarlo con las subtipologías asignadas
MAPA_TIPO_gdf['R_TAM'] = MAPA_TIPO_gdf.apply(asignar_subtipologia, axis=1)

# Crear el campo SUBTIPOLOGÍA
MAPA_TIPO_gdf['SUBTIPOLOGÍA_ue'] = MAPA_TIPO_gdf['TIPOLOGIA_ue'].astype(str) + '-' + MAPA_TIPO_gdf['R_TAM'].astype(str)
print(12)

# Guardar el GeoPackage con los nuevos campos
MAPA_TIPO_gdf.to_file(MAPA_TIPO_ruta, layer='Mzn_SubTipologias_ue', driver='GPKG')

# Resultados por Metrópoli

In [32]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Polygon
from dbfread import DBF
import jenkspy
import json
from jenkspy import jenks_breaks
import openpyxl
from openpyxl.styles import Font, PatternFill

#ruta gpkg fin
MAPA_TIPO_ruta = r"C:\Users\Alejandro\Downloads\DATA\Tratados\NAC\DENUE_LOC_NAC\Loc_Tipologías_NAC.gpkg"

# Leer la capa existente del GeoPackage base
MAPA_TIPO_gdf = gpd.read_file(MAPA_TIPO_ruta, layer='Loc_SubTipologias_ue')

In [33]:
# Filtrar polígonos por categoría 'Urbana' y 'Rural' con CALIF_CLAS diferente de '0'
MAPA_TIPO_urbana = MAPA_TIPO_gdf[MAPA_TIPO_gdf['AMBITO'] == 'Urbana']
MAPA_TIPO_rural = MAPA_TIPO_gdf[(MAPA_TIPO_gdf['AMBITO'] == 'Rural') & (MAPA_TIPO_gdf['CALIF_CLAS'] != '0')]

# Concatenar los GeoDataFrames filtrados
MAPA_TIPO_filtrado = pd.concat([MAPA_TIPO_urbana, MAPA_TIPO_rural])

# Columnas que se sumarán
columnas_a_sumar = ['POPTOT2010', 'POPTOT2020', 'VIVTOT2010', 'VIVTOT2020', 
    'TVIVHAB2010', 'TVIVHAB2020', 'TVIVPAR2010', 'TVIVPAR2020',
    'TVIVPARHAB2010', 'TVIVPARHAB2020', 'VIVPAR_HAB2010', 'VIVPAR_HAB2020',
    'VIVPAR_UT2010', 'VIVPAR_UT2020', 'OCUPVIVPAR2010', 'OCUPVIVPAR2020',
    'VIVPAR_DES2010', 'VIVPAR_DES2020', 'VIV1C2010', 'VIV1C2020', 
    'VIV2C2010', 'VIV2C2020', 'VIV3C2010', 'VIV3C2020', 'area_ha',
    'UE_20', 'Emp_20', 'UE_10', 'Emp_10']

# Sumar las columnas y agrupar por 'NOM_MET' y 'TIPO_MET'
MAPA_TIPO_sumado = MAPA_TIPO_filtrado.groupby(['CVE_MET','NOM_MET', 'TIPO_MET'])[columnas_a_sumar].sum(numeric_only=True).reset_index()

# Contar las localidades agrupadas
MAPA_TIPO_sumado['Localidades'] = MAPA_TIPO_filtrado.groupby(['NOM_MET', 'TIPO_MET']).size().values

# Recalcular los campos
MAPA_TIPO_sumado['DIFPOP'] = MAPA_TIPO_sumado['POPTOT2020'] - MAPA_TIPO_sumado['POPTOT2010']
MAPA_TIPO_sumado['DIFVIV'] = MAPA_TIPO_sumado['VIVTOT2020'] - MAPA_TIPO_sumado['VIVTOT2010']
MAPA_TIPO_sumado['RELPOP'] = MAPA_TIPO_sumado['DIFPOP'] / MAPA_TIPO_sumado['POPTOT2010']
MAPA_TIPO_sumado['RELVIV'] = MAPA_TIPO_sumado['DIFVIV'] / MAPA_TIPO_sumado['VIVTOT2010']
MAPA_TIPO_sumado['CVEPOP'] = np.where(MAPA_TIPO_sumado['RELPOP'] < -0.000000001, 'A', 'B')
MAPA_TIPO_sumado['CVEVIV'] = np.where(MAPA_TIPO_sumado['RELVIV'] < -.0703, '1',
                                      np.where((MAPA_TIPO_sumado['RELVIV'] >= -.0703) & (MAPA_TIPO_sumado['RELVIV'] <= .0703), 
                                               '2', '3'))
MAPA_TIPO_sumado['CVE'] = MAPA_TIPO_sumado['CVEPOP'] + MAPA_TIPO_sumado['CVEVIV'].astype(str)

conditions = [
    (MAPA_TIPO_sumado['POPTOT2020'] == 0),
    (MAPA_TIPO_sumado['POPTOT2010'] == 0) & (MAPA_TIPO_sumado['VIVTOT2010'] == 0) & (MAPA_TIPO_sumado['CVE'] == 'B3')
]
choices = ['DH', 'B3\'']
MAPA_TIPO_sumado['TIPOLOGIA'] = np.select(conditions, choices, default=MAPA_TIPO_sumado['CVE'])

MAPA_TIPO_sumado['DENS_POP10'] = MAPA_TIPO_sumado['POPTOT2010'] / MAPA_TIPO_sumado['area_ha']
MAPA_TIPO_sumado['DENS_POP20'] = MAPA_TIPO_sumado['POPTOT2020'] / MAPA_TIPO_sumado['area_ha']
MAPA_TIPO_sumado['DIF_DENSPOP'] = MAPA_TIPO_sumado['DENS_POP20'] - MAPA_TIPO_sumado['DENS_POP10']

MAPA_TIPO_sumado['DENS_VIV10'] = MAPA_TIPO_sumado['VIVTOT2010'] / MAPA_TIPO_sumado['area_ha']
MAPA_TIPO_sumado['DENS_VIV20'] = MAPA_TIPO_sumado['VIVTOT2020'] / MAPA_TIPO_sumado['area_ha']
MAPA_TIPO_sumado['DIF_DENSVIV'] = MAPA_TIPO_sumado['DENS_VIV20'] - MAPA_TIPO_sumado['DENS_VIV10']

MAPA_TIPO_sumado['CUARTOS2010'] = ((MAPA_TIPO_sumado['VIV1C2010'] * 1) + (MAPA_TIPO_sumado['VIV2C2010'] * 2) + (MAPA_TIPO_sumado['VIV3C2010'] * 4))
MAPA_TIPO_sumado['CUARTOS2020'] = ((MAPA_TIPO_sumado['VIV1C2020'] * 1) + (MAPA_TIPO_sumado['VIV2C2020'] * 2) + (MAPA_TIPO_sumado['VIV3C2020'] * 4))

MAPA_TIPO_sumado['CPERC2010'] = MAPA_TIPO_sumado['CUARTOS2010'] / MAPA_TIPO_sumado['POPTOT2010']
MAPA_TIPO_sumado['CPERC2020'] = MAPA_TIPO_sumado['CUARTOS2020'] / MAPA_TIPO_sumado['POPTOT2020']
MAPA_TIPO_sumado['DIF_CPERC'] = MAPA_TIPO_sumado['CPERC2020'] - MAPA_TIPO_sumado['CPERC2010']

MAPA_TIPO_sumado['HABXVIV2010'] = MAPA_TIPO_sumado['POPTOT2010'] / MAPA_TIPO_sumado['VIVTOT2010']
MAPA_TIPO_sumado['HABXVIV2020'] = MAPA_TIPO_sumado['POPTOT2020'] / MAPA_TIPO_sumado['VIVTOT2020']
MAPA_TIPO_sumado['DIF_HABXVIV'] = MAPA_TIPO_sumado['HABXVIV2020'] - MAPA_TIPO_sumado['HABXVIV2010']


MAPA_TIPO_sumado['DESHABITACION_2010'] = MAPA_TIPO_sumado['VIVPAR_DES2010'] / MAPA_TIPO_sumado['VIVTOT2010']
MAPA_TIPO_sumado['DESHABITACION_2020'] = MAPA_TIPO_sumado['VIVPAR_DES2020'] / MAPA_TIPO_sumado['VIVTOT2020']
MAPA_TIPO_sumado['DIF_DESHAB'] = MAPA_TIPO_sumado['DESHABITACION_2020'] - MAPA_TIPO_sumado['DESHABITACION_2010']

# Calcular TASA_POP_MUN
MAPA_TIPO_sumado['TASA_POP'] = (((MAPA_TIPO_sumado['POPTOT2020'] / MAPA_TIPO_sumado['POPTOT2010']) ** (1/10)) - 1) * 100
MAPA_TIPO_sumado['TASA_VIV'] = (((MAPA_TIPO_sumado['VIVTOT2020'] / MAPA_TIPO_sumado['VIVTOT2010']) ** (1/10)) - 1) * 100

def asignar_subtipologia_2010(row):
    # Valores específicos para deshabitación
    rangos_deshabitacion_grupo = [0, .0703, .1416, .2812]  # Modifica estos valores según tus necesidades
    
    # Asignar subtipología según los rangos definidos
    if rangos_deshabitacion_grupo[0] <= row['DESHABITACION_2010'] < rangos_deshabitacion_grupo[1]:
        return 'a'
    elif rangos_deshabitacion_grupo[1] <= row['DESHABITACION_2010'] < rangos_deshabitacion_grupo[2]:
        return 'b'
    elif rangos_deshabitacion_grupo[2] <= row['DESHABITACION_2010'] < rangos_deshabitacion_grupo[3]:
        return 'c'
    elif row['DESHABITACION_2010'] >= rangos_deshabitacion_grupo[3]:
        return 'd'
    else:
        return '0'  # Valor predeterminado

    
def asignar_subtipologia_2020(row):
    # Valores específicos para deshabitación
    rangos_deshabitacion_grupo = [0, .0703, .1416, .2812]  # Modifica estos valores según tus necesidades
    
    # Asignar subtipología según los rangos definidos
    if rangos_deshabitacion_grupo[0] <= row['DESHABITACION_2020'] < rangos_deshabitacion_grupo[1]:
        return 'a'
    elif rangos_deshabitacion_grupo[1] <= row['DESHABITACION_2020'] < rangos_deshabitacion_grupo[2]:
        return 'b'
    elif rangos_deshabitacion_grupo[2] <= row['DESHABITACION_2020'] < rangos_deshabitacion_grupo[3]:
        return 'c'
    elif row['DESHABITACION_2020'] >= rangos_deshabitacion_grupo[3]:
        return 'd'
    else:
        return '0'  # Valor predeterminado


# Crear un nuevo campo 'R_DESHAB' y llenarlo con las subtipologías asignadas
MAPA_TIPO_sumado['R_DESHAB_2010'] = MAPA_TIPO_sumado.apply(asignar_subtipologia_2010, axis=1)

# Crear un nuevo campo 'R_DESHAB' y llenarlo con las subtipologías asignadas
MAPA_TIPO_sumado['R_DESHAB_2020'] = MAPA_TIPO_sumado.apply(asignar_subtipologia_2020, axis=1)

# Crear el campo SUBTIPOLOGÍA
MAPA_TIPO_sumado['SUBTIPOLOGÍA'] = MAPA_TIPO_sumado['TIPOLOGIA'].astype(str) + '-' + MAPA_TIPO_sumado['R_DESHAB_2020'].astype(str)

############################################indicadores DENUE

#Tamaño de unidades promedio: Empleados entre UE por manzana
MAPA_TIPO_sumado['TAMAÑO'] = MAPA_TIPO_sumado['Emp_20'] / MAPA_TIPO_sumado['UE_20']

# Calcular las sumas de POPTOT2020 y POPTOT2010 por CVE_MET
sum_emptot2020 = MAPA_TIPO_sumado.groupby('CVE_MET')['Emp_20'].transform('sum')
sum_emptot2010 = MAPA_TIPO_sumado.groupby('CVE_MET')['Emp_10'].transform('sum')

# Calcular TASA_POP_MUN
MAPA_TIPO_sumado['TASA_EMP_MET'] = (((sum_emptot2020 / sum_emptot2010) ** (1/10)) - 1) * 100

#resto de campos
MAPA_TIPO_sumado['DIF_EMP'] = MAPA_TIPO_sumado['Emp_20'] - MAPA_TIPO_sumado['Emp_10']
MAPA_TIPO_sumado['DIF_UEC'] = MAPA_TIPO_sumado['UE_20'] - MAPA_TIPO_sumado['UE_10']
MAPA_TIPO_sumado['REL_EMP'] = MAPA_TIPO_sumado['DIF_EMP'] / MAPA_TIPO_sumado['Emp_10']
MAPA_TIPO_sumado['REL_UEC'] = MAPA_TIPO_sumado['DIF_UEC'] / MAPA_TIPO_sumado['UE_10']
MAPA_TIPO_sumado['CVE_EMP'] = np.where(MAPA_TIPO_sumado['REL_EMP'] < -0.000000001, '1', '2')
MAPA_TIPO_sumado['CVE_UEC'] = np.where(MAPA_TIPO_sumado['REL_UEC'] < -.1, 'A',
                                   np.where((MAPA_TIPO_sumado['REL_UEC'] >= -.1) & (MAPA_TIPO_sumado['REL_UEC'] <= .1), 
                                            'B', 'C'))
MAPA_TIPO_sumado['CVE_ue'] = MAPA_TIPO_sumado['CVE_UEC'].astype(str) + MAPA_TIPO_sumado['CVE_EMP']
conditions = [
    (MAPA_TIPO_sumado['Emp_20'] == 0),
    (MAPA_TIPO_sumado['Emp_10'] == 0) & (MAPA_TIPO_sumado['UE_10'] == 0) & (MAPA_TIPO_sumado['CVE_ue'] == 'C2')
]
choices = ['00', 'C2+']
MAPA_TIPO_sumado['TIPOLOGIA_ue'] = np.select(conditions, choices, default=MAPA_TIPO_sumado['CVE_ue'])

MAPA_TIPO_sumado['DENS_EMP20'] = MAPA_TIPO_sumado['Emp_20'] / MAPA_TIPO_sumado['area_ha']
MAPA_TIPO_sumado['DENS_EMP10'] = MAPA_TIPO_sumado['Emp_10'] / MAPA_TIPO_sumado['area_ha']

MAPA_TIPO_sumado['DENS_UEC20'] = MAPA_TIPO_sumado['UE_20'] / MAPA_TIPO_sumado['area_ha']
MAPA_TIPO_sumado['DENS_UEC10'] = MAPA_TIPO_sumado['UE_10'] / MAPA_TIPO_sumado['area_ha']


def asignar_subtipologia(row):
    # Valores específicos para deshabitación
    rangos_deshabitacion_grupo = [0, 1, 30, 100]  # Modifica estos valores según tus necesidades
    
# Asignar subtipología según los rangos definidos
    if rangos_deshabitacion_grupo[0] <= row['TAMAÑO'] < rangos_deshabitacion_grupo[1]:
        return 'SUE'
    elif rangos_deshabitacion_grupo[1] <= row['TAMAÑO'] < rangos_deshabitacion_grupo[2]:
        return 'MICRO'
    elif rangos_deshabitacion_grupo[2] <= row['TAMAÑO'] < rangos_deshabitacion_grupo[3]:
        return 'MEDIANA'
    elif row['TAMAÑO'] >= rangos_deshabitacion_grupo[3]:
        return 'GRANDE'
    else:
        return '0'  # Valor predeterminado

# Crear un nuevo campo 'R_DESHAB' y llenarlo con las subtipologías asignadas
MAPA_TIPO_sumado['R_TAM'] = MAPA_TIPO_sumado.apply(asignar_subtipologia, axis=1)

# Crear el campo SUBTIPOLOGÍA
MAPA_TIPO_sumado['SUBTIPOLOGÍA_ue'] = MAPA_TIPO_sumado['TIPOLOGIA_ue'].astype(str) + '-' + MAPA_TIPO_sumado['R_TAM'].astype(str)

###################################################################

# Redondear a 3 decimales
MAPA_TIPO_sumado = MAPA_TIPO_sumado.round(3)

# Guardar los datos en un archivo Excel
excel_path = r"C:\Users\Alejandro\Downloads\DATA\Tratados\NAC\DENUE_LOC_NAC\resultados_metropoli.xlsx"
MAPA_TIPO_sumado.to_excel(excel_path, index=False)

# Leer el archivo Excel
wb = openpyxl.load_workbook(excel_path)

# Seleccionar la hoja de cálculo
sheet = wb.active

# Ajustar el ancho de las columnas automáticamente
for col in sheet.columns:
    max_length = 0
    column = col[0].column_letter
    for cell in col:
        try:
            if len(str(cell.value)) > max_length:
                max_length = len(cell.value)
        except:
            pass
    adjusted_width = (max_length + 2) * 1.2
    sheet.column_dimensions[column].width = adjusted_width

# Aplicar color rojo a las celdas negativas en columnas con prefijo 'DIF_'
red_font = Font(color="FF0000")

for col in sheet.iter_cols(min_row=2, min_col=1, max_col=sheet.max_column):
    col_name = sheet.cell(row=1, column=col[0].column).value
    if col_name.startswith('DIF_'):
        for cell in col:
            if isinstance(cell.value, (int, float)) and cell.value < 0:
                cell.font = red_font

# Guardar los cambios en el archivo Excel
wb.save(r"C:\Users\Alejandro\Downloads\DATA\Tratados\NAC\DENUE_LOC_NAC\estadísticas_bruto_metropoli.xlsx")

# Resultados por Municipio

In [ ]:
import pandas as pd
import numpy as np
import openpyxl
from openpyxl.styles import Font, PatternFill

# Filtrar polígonos por categoría 'Urbana' y 'Rural' con CALIF_CLAS diferente de '0'
MAPA_TIPO_urbana = MAPA_TIPO_gdf[MAPA_TIPO_gdf['AMBITO'] == 'Urbana']
MAPA_TIPO_rural = MAPA_TIPO_gdf[(MAPA_TIPO_gdf['AMBITO'] == 'Rural') & (MAPA_TIPO_gdf['CALIF_CLAS'] != '0')]

# Concatenar los GeoDataFrames filtrados
MAPA_TIPO_filtrado = pd.concat([MAPA_TIPO_urbana, MAPA_TIPO_rural])

# Columnas que se sumarán
columnas_a_sumar = ['POPTOT2010', 'POPTOT2020', 'VIVTOT2010', 'VIVTOT2020', 
    'TVIVHAB2010', 'TVIVHAB2020', 'TVIVPAR2010', 'TVIVPAR2020',
    'TVIVPARHAB2010', 'TVIVPARHAB2020', 'VIVPAR_HAB2010', 'VIVPAR_HAB2020',
    'VIVPAR_UT2010', 'VIVPAR_UT2020', 'OCUPVIVPAR2010', 'OCUPVIVPAR2020',
    'VIVPAR_DES2010', 'VIVPAR_DES2020', 'VIV1C2010', 'VIV1C2020', 
    'VIV2C2010', 'VIV2C2020', 'VIV3C2010', 'VIV3C2020', 'area_ha',
    'UE_20', 'Emp_20', 'UE_10', 'Emp_10']

# Sumar las columnas y agrupar por 'NOM_MET'
MAPA_TIPO_sumado = MAPA_TIPO_filtrado.groupby(['CVE_MET','CVEMUN','NOM_MET', 'TIPO_MET','TIPO_MUN'])[columnas_a_sumar].sum(numeric_only=True).reset_index()

# Contar las manzanas agrupadas
MAPA_TIPO_sumado['Localidades'] = MAPA_TIPO_filtrado.groupby(['CVEMUN', 'NOM_MET', 'TIPO_MET','TIPO_MUN']).size().values



# # Intentar cargar el archivo CSV con diferentes codificaciones y asegurar que las columnas sean string
# try:
#     agem_df = pd.read_csv("C:/Users/ADMIN/Downloads/AGEM.csv", usecols=['CVEMUN', 'NOM_MUN'], dtype=str, encoding='latin1')
# except UnicodeDecodeError:
#     agem_df = pd.read_csv("C:/Users/ADMIN/Downloads/AGEM.csv", usecols=['CVEMUN', 'NOM_MUN'], dtype=str, encoding='iso-8859-1')

# # Hacer el join con el archivo CSV basado en 'CVEMUN'
# MAPA_TIPO_sumado = pd.merge(MAPA_TIPO_sumado, agem_df, on='CVEMUN', how='left')


# Recalcular los campos
MAPA_TIPO_sumado['DIFPOP'] = MAPA_TIPO_sumado['POPTOT2020'] - MAPA_TIPO_sumado['POPTOT2010']
MAPA_TIPO_sumado['DIFVIV'] = MAPA_TIPO_sumado['VIVTOT2020'] - MAPA_TIPO_sumado['VIVTOT2010']
MAPA_TIPO_sumado['RELPOP'] = MAPA_TIPO_sumado['DIFPOP'] / MAPA_TIPO_sumado['POPTOT2010']
MAPA_TIPO_sumado['RELVIV'] = MAPA_TIPO_sumado['DIFVIV'] / MAPA_TIPO_sumado['VIVTOT2010']
MAPA_TIPO_sumado['CVEPOP'] = np.where(MAPA_TIPO_sumado['RELPOP'] < -0.000000001, 'A', 'B')
MAPA_TIPO_sumado['CVEVIV'] = np.where(MAPA_TIPO_sumado['RELVIV'] < -.0703, '1',
                                      np.where((MAPA_TIPO_sumado['RELVIV'] >= -.0703) & (MAPA_TIPO_sumado['RELVIV'] <= .0703), 
                                               '2', '3'))
MAPA_TIPO_sumado['CVE'] = MAPA_TIPO_sumado['CVEPOP'] + MAPA_TIPO_sumado['CVEVIV'].astype(str)

conditions = [
    (MAPA_TIPO_sumado['POPTOT2020'] == 0),
    (MAPA_TIPO_sumado['POPTOT2010'] == 0) & (MAPA_TIPO_sumado['VIVTOT2010'] == 0) & (MAPA_TIPO_sumado['CVE'] == 'B3')
]
choices = ['DH', 'B3\'']
MAPA_TIPO_sumado['TIPOLOGIA'] = np.select(conditions, choices, default=MAPA_TIPO_sumado['CVE'])

MAPA_TIPO_sumado['DENS_POP10'] = MAPA_TIPO_sumado['POPTOT2010'] / MAPA_TIPO_sumado['area_ha']
MAPA_TIPO_sumado['DENS_POP20'] = MAPA_TIPO_sumado['POPTOT2020'] / MAPA_TIPO_sumado['area_ha']
MAPA_TIPO_sumado['DIF_DENSPOP'] = MAPA_TIPO_sumado['DENS_POP20'] - MAPA_TIPO_sumado['DENS_POP10']

MAPA_TIPO_sumado['DENS_VIV10'] = MAPA_TIPO_sumado['VIVTOT2010'] / MAPA_TIPO_sumado['area_ha']
MAPA_TIPO_sumado['DENS_VIV20'] = MAPA_TIPO_sumado['VIVTOT2020'] / MAPA_TIPO_sumado['area_ha']
MAPA_TIPO_sumado['DIF_DENSVIV'] = MAPA_TIPO_sumado['DENS_VIV20'] - MAPA_TIPO_sumado['DENS_VIV10']

MAPA_TIPO_sumado['CUARTOS2010'] = ((MAPA_TIPO_sumado['VIV1C2010'] * 1) + (MAPA_TIPO_sumado['VIV2C2010'] * 2) + (MAPA_TIPO_sumado['VIV3C2010'] * 4))
MAPA_TIPO_sumado['CUARTOS2020'] = ((MAPA_TIPO_sumado['VIV1C2020'] * 1) + (MAPA_TIPO_sumado['VIV2C2020'] * 2) + (MAPA_TIPO_sumado['VIV3C2020'] * 4))

MAPA_TIPO_sumado['CPERC2010'] = MAPA_TIPO_sumado['CUARTOS2010'] / MAPA_TIPO_sumado['POPTOT2010']
MAPA_TIPO_sumado['CPERC2020'] = MAPA_TIPO_sumado['CUARTOS2020'] / MAPA_TIPO_sumado['POPTOT2020']
MAPA_TIPO_sumado['DIF_CPERC'] = MAPA_TIPO_sumado['CPERC2020'] - MAPA_TIPO_sumado['CPERC2010']

MAPA_TIPO_sumado['HABXVIV2010'] = MAPA_TIPO_sumado['POPTOT2010'] / MAPA_TIPO_sumado['VIVTOT2010']
MAPA_TIPO_sumado['HABXVIV2020'] = MAPA_TIPO_sumado['POPTOT2020'] / MAPA_TIPO_sumado['VIVTOT2020']
MAPA_TIPO_sumado['DIF_HABXVIV'] = MAPA_TIPO_sumado['HABXVIV2020'] - MAPA_TIPO_sumado['HABXVIV2010']


MAPA_TIPO_sumado['DESHABITACION_2010'] = MAPA_TIPO_sumado['VIVPAR_DES2010'] / MAPA_TIPO_sumado['VIVTOT2010']
MAPA_TIPO_sumado['DESHABITACION_2020'] = MAPA_TIPO_sumado['VIVPAR_DES2020'] / MAPA_TIPO_sumado['VIVTOT2020']
MAPA_TIPO_sumado['DIF_DESHAB'] = MAPA_TIPO_sumado['DESHABITACION_2020'] - MAPA_TIPO_sumado['DESHABITACION_2010']

# Calcular TASA_POP_MUN
MAPA_TIPO_sumado['TASA_POP'] = (((MAPA_TIPO_sumado['POPTOT2020'] / MAPA_TIPO_sumado['POPTOT2010']) ** (1/10)) - 1) * 100
MAPA_TIPO_sumado['TASA_VIV'] = (((MAPA_TIPO_sumado['VIVTOT2020'] / MAPA_TIPO_sumado['VIVTOT2010']) ** (1/10)) - 1) * 100

def asignar_subtipologia_2010(row):
    # Valores específicos para deshabitación
    rangos_deshabitacion_grupo = [0, .0703, .1416, .2812]  # Modifica estos valores según tus necesidades
    
    # Asignar subtipología según los rangos definidos
    if rangos_deshabitacion_grupo[0] <= row['DESHABITACION_2010'] < rangos_deshabitacion_grupo[1]:
        return 'a'
    elif rangos_deshabitacion_grupo[1] <= row['DESHABITACION_2010'] < rangos_deshabitacion_grupo[2]:
        return 'b'
    elif rangos_deshabitacion_grupo[2] <= row['DESHABITACION_2010'] < rangos_deshabitacion_grupo[3]:
        return 'c'
    elif row['DESHABITACION_2010'] >= rangos_deshabitacion_grupo[3]:
        return 'd'
    else:
        return '0'  # Valor predeterminado

    
def asignar_subtipologia_2020(row):
    # Valores específicos para deshabitación
    rangos_deshabitacion_grupo = [0, .0703, .1416, .2812]  # Modifica estos valores según tus necesidades
    
    # Asignar subtipología según los rangos definidos
    if rangos_deshabitacion_grupo[0] <= row['DESHABITACION_2020'] < rangos_deshabitacion_grupo[1]:
        return 'a'
    elif rangos_deshabitacion_grupo[1] <= row['DESHABITACION_2020'] < rangos_deshabitacion_grupo[2]:
        return 'b'
    elif rangos_deshabitacion_grupo[2] <= row['DESHABITACION_2020'] < rangos_deshabitacion_grupo[3]:
        return 'c'
    elif row['DESHABITACION_2020'] >= rangos_deshabitacion_grupo[3]:
        return 'd'
    else:
        return '0'  # Valor predeterminado


# Crear un nuevo campo 'R_DESHAB' y llenarlo con las subtipologías asignadas
MAPA_TIPO_sumado['R_DESHAB_2010'] = MAPA_TIPO_sumado.apply(asignar_subtipologia_2010, axis=1)

# Crear un nuevo campo 'R_DESHAB' y llenarlo con las subtipologías asignadas
MAPA_TIPO_sumado['R_DESHAB_2020'] = MAPA_TIPO_sumado.apply(asignar_subtipologia_2020, axis=1)

# Crear el campo SUBTIPOLOGÍA
MAPA_TIPO_sumado['SUBTIPOLOGÍA'] = MAPA_TIPO_sumado['TIPOLOGIA'].astype(str) + '-' + MAPA_TIPO_sumado['R_DESHAB_2020'].astype(str)

############################################indicadores DENUE

#Tamaño de unidades promedio: Empleados entre UE por manzana
MAPA_TIPO_sumado['TAMAÑO'] = MAPA_TIPO_sumado['Emp_20'] / MAPA_TIPO_sumado['UE_20']

# Calcular las sumas de POPTOT2020 y POPTOT2010 por CVE_MET
sum_emptot2020 = MAPA_TIPO_sumado.groupby('CVEMUN')['Emp_20'].transform('sum')
sum_emptot2010 = MAPA_TIPO_sumado.groupby('CVEMUN')['Emp_10'].transform('sum')

# Calcular TASA_POP_MUN
MAPA_TIPO_sumado['TASA_EMP_MUN'] = (((sum_emptot2020 / sum_emptot2010) ** (1/10)) - 1) * 100

#resto de campos
MAPA_TIPO_sumado['DIF_EMP'] = MAPA_TIPO_sumado['Emp_20'] - MAPA_TIPO_sumado['Emp_10']
MAPA_TIPO_sumado['DIF_UEC'] = MAPA_TIPO_sumado['UE_20'] - MAPA_TIPO_sumado['UE_10']
MAPA_TIPO_sumado['REL_EMP'] = MAPA_TIPO_sumado['DIF_EMP'] / MAPA_TIPO_sumado['Emp_10']
MAPA_TIPO_sumado['REL_UEC'] = MAPA_TIPO_sumado['DIF_UEC'] / MAPA_TIPO_sumado['UE_10']
MAPA_TIPO_sumado['CVE_EMP'] = np.where(MAPA_TIPO_sumado['REL_EMP'] < -0.000000001, '1', '2')
MAPA_TIPO_sumado['CVE_UEC'] = np.where(MAPA_TIPO_sumado['REL_UEC'] < -.1, 'A',
                                   np.where((MAPA_TIPO_sumado['REL_UEC'] >= -.1) & (MAPA_TIPO_sumado['REL_UEC'] <= .1), 
                                            'B', 'C'))
MAPA_TIPO_sumado['CVE_ue'] = MAPA_TIPO_sumado['CVE_UEC'].astype(str) + MAPA_TIPO_sumado['CVE_EMP']
conditions = [
    (MAPA_TIPO_sumado['Emp_20'] == 0),
    (MAPA_TIPO_sumado['Emp_10'] == 0) & (MAPA_TIPO_sumado['UE_10'] == 0) & (MAPA_TIPO_sumado['CVE_ue'] == 'C2')
]
choices = ['00', 'C2+']
MAPA_TIPO_sumado['TIPOLOGIA_ue'] = np.select(conditions, choices, default=MAPA_TIPO_sumado['CVE_ue'])

MAPA_TIPO_sumado['DENS_EMP20'] = MAPA_TIPO_sumado['Emp_20'] / MAPA_TIPO_sumado['area_ha']
MAPA_TIPO_sumado['DENS_EMP10'] = MAPA_TIPO_sumado['Emp_10'] / MAPA_TIPO_sumado['area_ha']

MAPA_TIPO_sumado['DENS_UEC20'] = MAPA_TIPO_sumado['UE_20'] / MAPA_TIPO_sumado['area_ha']
MAPA_TIPO_sumado['DENS_UEC10'] = MAPA_TIPO_sumado['UE_10'] / MAPA_TIPO_sumado['area_ha']


def asignar_subtipologia(row):
    # Valores específicos para deshabitación
    rangos_deshabitacion_grupo = [0, 1, 30, 100]  # Modifica estos valores según tus necesidades
    
# Asignar subtipología según los rangos definidos
    if rangos_deshabitacion_grupo[0] <= row['TAMAÑO'] < rangos_deshabitacion_grupo[1]:
        return 'SUE'
    elif rangos_deshabitacion_grupo[1] <= row['TAMAÑO'] < rangos_deshabitacion_grupo[2]:
        return 'MICRO'
    elif rangos_deshabitacion_grupo[2] <= row['TAMAÑO'] < rangos_deshabitacion_grupo[3]:
        return 'MEDIANA'
    elif row['TAMAÑO'] >= rangos_deshabitacion_grupo[3]:
        return 'GRANDE'
    else:
        return '0'  # Valor predeterminado

# Crear un nuevo campo 'R_DESHAB' y llenarlo con las subtipologías asignadas
MAPA_TIPO_sumado['R_TAM'] = MAPA_TIPO_sumado.apply(asignar_subtipologia, axis=1)

# Crear el campo SUBTIPOLOGÍA
MAPA_TIPO_sumado['SUBTIPOLOGÍA_ue'] = MAPA_TIPO_sumado['TIPOLOGIA_ue'].astype(str) + '-' + MAPA_TIPO_sumado['R_TAM'].astype(str)

###################################################################

# Redondear a 3 decimales
MAPA_TIPO_sumado = MAPA_TIPO_sumado.round(3)

# Reordenar columnas: 'NOM_MUN', 'NOM_MET', 'TIPO_MET' al inicio y 'TIPO_MUN' al final
column_order = ['CVEMUN', 'NOM_MET', 'TIPO_MET'] + [col for col in MAPA_TIPO_sumado.columns if col not in ['CVEMUN', 'NOM_MET', 'TIPO_MET', 'TIPO_MUN']] + ['TIPO_MUN']
MAPA_TIPO_sumado = MAPA_TIPO_sumado[column_order]

# Guardar los datos en un archivo Excel
# excel_path = 'C:/Users/ADMIN/Desktop/Ejercicio/Localidad/resultados_municipio.xlsx'
excel_path = r"C:\Users\Alejandro\Downloads\DATA\Tratados\NAC\DENUE_LOC_NAC\resultados_municipio.xlsx"
MAPA_TIPO_sumado.to_excel(excel_path, index=False)

# Leer el archivo Excel
wb = openpyxl.load_workbook(excel_path)

# Seleccionar la hoja de cálculo
sheet = wb.active

# Ajustar el ancho de las columnas automáticamente
for col in sheet.columns:
    max_length = 0
    column = col[0].column_letter
    for cell in col:
        try:
            if len(str(cell.value)) > max_length:
                max_length = len(cell.value)
        except:
            pass
    adjusted_width = (max_length + 2) * 1.2
    sheet.column_dimensions[column].width = adjusted_width

# Aplicar color rojo a las celdas negativas en columnas con prefijo 'DIF_'
red_font = Font(color="FF0000")

for col in sheet.iter_cols(min_row=2, min_col=1, max_col=sheet.max_column):
    col_name = sheet.cell(row=1, column=col[0].column).value
    if col_name.startswith('DIF_'):
        for cell in col:
            if isinstance(cell.value, (int, float)) and cell.value < 0:
                cell.font = red_font

# Guardar los cambios en el archivo Excel
# wb.save('C:/Users/ADMIN/Desktop/Ejercicio/Localidad/estadísticas_bruto_municipio.xlsx')
wb.save(r"C:\Users\Alejandro\Downloads\DATA\Tratados\NAC\DENUE_LOC_NAC\estadísticas_bruto_municipio.xlsx")

# Combinar bruto y neto

In [1]:
import pandas as pd
import numpy as np
import openpyxl
from openpyxl.styles import Font, PatternFill
from openpyxl.utils import get_column_letter

# Rutas de los archivos
ruta_bruto = r"C:\Users\ADMIN\Desktop\Ejercicio\Localidad\estadísticas_bruto_metrópoli.xlsx"
ruta_neto = r"C:\Users\ADMIN\OneDrive - Universidad de Guadalajara\3. Voces\2024\Tipologías\Nacional\estadísticas_por_metropoli.xlsx"
ruta_salida = r"C:\Users\ADMIN\Desktop\Ejercicio\Localidad\estadísticas_unidas_metrópoli.xlsx"

In [2]:
# Cargar los archivos Excel
df_bruto = pd.read_excel(ruta_bruto)
df_neto = pd.read_excel(ruta_neto)

# Añadir prefijos a las columnas
df_bruto = df_bruto.add_prefix('bruto_')
df_neto = df_neto.add_prefix('neto_')

# Cambiar el nombre de la columna 'bruto_NOM_MET' a 'NOM_MET' para realizar el merge
df_bruto = df_bruto.rename(columns={'bruto_NOM_MET': 'NOM_MET'})
df_neto = df_neto.rename(columns={'neto_NOM_MET': 'NOM_MET'})

# Realizar la unión por 'NOM_MET'
df_merged = pd.merge(df_bruto, df_neto, on='NOM_MET', how='outer')

# Reordenar las columnas
cols = sorted(df_merged.columns, key=lambda x: (x.split('_')[-1], x))
df_merged = df_merged[['NOM_MET'] + [col for col in cols if col != 'NOM_MET']]

# Guardar el DataFrame resultante a un nuevo archivo Excel
df_merged.to_excel(ruta_salida, index=False)

# Ajustar el tamaño de las columnas y aplicar estilos
wb = openpyxl.load_workbook(ruta_salida)
ws = wb.active

# Ajustar el tamaño de las columnas basado en el nombre del campo
for col in ws.columns:
    max_length = 0
    column = col[0].column_letter # Get the column name
    for cell in col:
        try:
            if len(str(cell.value)) > max_length:
                max_length = len(cell.value)
        except:
            pass
    adjusted_width = (max_length + 2)
    ws.column_dimensions[column].width = adjusted_width

# Cambiar la fuente a Roboto y colorear valores menores a 0
roboto_font = Font(name='Roboto')

for row in ws.iter_rows(min_row=2):
    for cell in row:
        cell.font = roboto_font
        if cell.value is not None and isinstance(cell.value, (int, float)) and cell.value < 0:
            cell.font = Font(color="FF0000", name='Roboto')

# Guardar los cambios en el archivo Excel
wb.save(ruta_salida)